# Homework: Spark SQL

In this homework you will gain a mastery of using Spark SQL. By the end, you'll be a star (not that you aren't already one). Over the next few days you will be using an EMR cluster to use Spark to manipulate the entire `linkedin.json` dataset as well as a new data set, `stock_prices.csv`.

The goal of the homework will be to create a training dataset for a Random Forest Machine learning model. The training data set will contain the monthly number of employees hired by companies in `linkedin.json` and their corresponding closing stock prices over a 10 year period (2000-2011). We will try and predict, based on this data, if the company will have a positive or negative growth in stock in the first quarter of the next year. Who's ready to make some money?

## The Noteworthy Notes
Before we begin here are some important notes to keep in mind,


1.   **IMPORTANT!** I said it twice, it's really important. In this homework, we will be using AWS resources. You are given a quota ($150) to use for the entirety of the homework. There is a small chance you will use all this money, however it is important that at the end of every session, you **shut down your EMR cluster**.
2.   **You can only use Google Colab for this Homework** since we must connect to the EMR cluster and Jupyter doesn't do that. Using a Google Colab Notebook with an EMR cluster has two important abnormalities:
    * The first line of any cell in which you will use the spark session must be `%%spark`. Notice that all cells below have this.
    * You will, unfortunately, not be able to stop a cell while it is running. If you wish to do so, you will need to restart your cluster. See the Setup EMR Document for reference.
3.   You are **required** to use Spark SQL queries to handle the data in the assignment. Mastering SQL is more beneficial than being able to use Spark commands (functions) as it will show up in more areas of programming and data science/analytics than just Spark. Use the following [function list](https://spark.apache.org/docs/latest/api/sql/index.html#) to see all the SQL functions avaliable in Spark.
4. There are portions of this homework that are _very_ challenging. 

With that said, let's dive in.




## Step 0: Set up EMR

Your first task is to create an EMR cluster your AWS Educate Accounts. Please see the [attached document](https://drive.google.com/open?id=1_8NB_3QXfQKm5Vyu7XyxU2mnH5HnGM-F) for detailed instructions. Move on to Step 0.1 after you have completed all the steps in the document.

### Step 0.1: The Superfluous Setup

Run the following two cells. These will allow your colab notebook to connect to an use your EMR.

In [ ]:
%%capture
!apt update
!apt install gcc python-dev libkrb5-dev
!pip install sparkmagic
!pip install pyspark

In [ ]:
%%capture
%load_ext sparkmagic.magics 

In [ ]:
from pyspark.sql.types import *
import json
import urllib.request

from datetime import datetime

### Step 0.2: The Succulent Spark

Now, connect your notebook to the EMR cluster you created. In the first cell, copy the link to the Master Public DNS specified in the setup document. You will need to add `http://` to the beginning of the address and the port to the end. The final format should be,

`http://<your-DNS-link>:8998`

For example, if my DNS (directly from the AWS EMR console) is `ec2-3-15-237-211.us-east-2.compute.amazonaws.com` my address would be,

`http://ec2-3-15-237-211.us-east-2.compute.amazonaws.com:8998`

Insert this in the `# TODO # below`. For our example, the cell would read,



```
%spark add -s spark_session -l python -u http://ec2-3-15-237-211.us-east-2.compute.amazonaws.com:8998
```

In [ ]:
# TODO: Enter your Master Public DNS with the proper formatting and host

%spark add -s spark_session -l python -u http://ec2-34-205-85-50.compute-1.amazonaws.com:8998 

In [ ]:
# To delte a session. 
# %spark delete -s spark_session_name

_Note_: Since we are using an EMR cluster we will not have access to the several modules that exist for python, e.g. `pandas`, `numpy`, etc. We have written the entire homework such that the solution does not require any of these. Don't try to import them. It won't work.

## Step 1: Data Cleaning and Shaping

The data you will use is stored in an S3 bucket, a cloud storage service. You now need to download it onto the nodes of your [EMR cluster](https://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-what-is-emr.html). 

### Step 1.1: The Stupendous Schema

When loading data, Spark will try to infer it's structure on it's own. This process is faulty because it will sometimes infer the type incorrectly. JSON documents, like the one we will use, can have nested types, such as: arrays, arrays of dictionaries, dictionaries of dictionaries, etc. Spark's ability to determine these nested types is not reliable, thus you will define a schema for `linkedin.json`.

A schema is a description of the structure of data. You will be defining an explicit schema for `linkedin.json`. In Spark, schema's are defined using a `StructType` object. This is a collection of data types, termed `StructField`'s, that specify the structure and variable type of each component of the dataset. For example, suppose we have the following simple JSON object,


```
{
 "student_name": "Leonardo Murri",
 "GPA": 1.4,
 "courses": [
    {"department": "Computer and Information Science",
     "course_id": "CIS 545",
     "semester": "Fall 2018"},
    {"department": "Computer and Information Science",
     "course_id": "CIS 520",
     "semester": "Fall 2018"},
    {"department": "Electrical and Systems Engineering",
     "course_id": "ESE 650",
     "semester": "Spring 2018"}
 ],
 "grad_year": 2019
 }
```

We would define it's schema as follows,

```       
schema = StructType([
           StructField("student_name", StringType(), nullable=True),
           StructField("GPA", FloatType(), nullable=True),
           StructField("courses", ArrayType(
                StructType([
                  StructField("department", StringType(), nullable=True),
                  StructField("course_id", StringType(), nullable=True),
                  StructField("semester", StringType(), nullable=True)
                ])
           ), nullable=True),
           StructField("grad_year", IntegerType(), nullable=True)
         ])
```


Each `StructField` has the following structure: `(name, type, nullable)`. The `nullable` flag defines that the specified field may be empty. Your first task is to define the `schema` of `linkedin.json`. A sample JSON object for the dataset can be found [here](http://oneclickpaste.com/194048/).

_Note_: In `linkedin.json` the field `specilities` is spelled incorrectly. This is **not** a typo. 



In [ ]:
%%spark

# TODO: Define [linkedin.json] schema
# YOUR CODE HERE


### Step 1.2: The Laudable Loading

Load the `linkedin.json` dataset from your S3 bucket into a Spark dataframe (sdf) called `raw_data_sdf`. If you have constructed `schema` correctly `spark.read.json()` will read in the dataset. ***You do not need to edit this cell***.

In [ ]:
%%spark
# TODO: modify the location to the place where you have the data. 
raw_data_sdf = spark.read.json("s3a://opends4allhw3/linkedin.json", schema=schema)
# raw_data_sdf = spark.read.json('linkedin.json', schema=schema)

The cell below shows how to run SQL commands on Spark tables. Use this as a template for all your SQL queries in this notebook. ***You do not need to edit this cell***.

In [ ]:
%%spark

# Create SQL-accesible table
raw_data_sdf.createOrReplaceTempView("raw_data")

# Declare SQL query to be excecuted
query = '''SELECT * 
           FROM raw_data'''

# Save the output sdf of spark.sql() as answer_sdf
answer_sdf = spark.sql(query)

# Display the first 10 rows
answer_sdf.show(10)

### Step 1.3: The Extravagent Extraction

In our training model, we are interested in when individuals began working at a company.  From creating the schema, you should notice that the collection of companies inviduals worked at are contained in the `experience` field as an array of dictionaries. You should use the `org` for the company name and `start` for the start date. Here is an example of an `experience` field,

```
{
   "experience": [
     {
        "org": "The Walt Disney Company", 
        "title" : "Mickey Mouse",
        "end" : "Present",
        "start": "November 1928",
        "desc": "Sailed a boat."
     },
     {
        "org": "Walt Disney World Resort",
        "title": "Mickey Mouse Mascot",
        "start": "January 2005",
        "desc": "Took pictures with kids."
     }
   ]
}
```

Your task is to extract each pair of company and start date from these arrays. In Spark, this is known as "exploding" a row. An explode will seperate the elements of an array into multiple rows.

Create an sdf called `raw_start_dates_sdf` that contains the company and start date for every experience of every individual in `raw_data_sdf`. Drop any row that contains a `null` in either column with `dropna()`. You can sort the elements however you wish (you don't need to if you don't want to). The sdf should look as follows:

```
+--------------------------+---------------+
|org                       |start_date     |
+--------------------------+---------------+
|Walt Disney World Resort  |January 2005   | 
|The Walt Disney Company   |November 1928  |
|...                       |...            |
+--------------------------+---------------+
```

_Hint_: You may want to do two seperate explodes for `org` and `start`. In an explode, the position of the element in the array can be extracted as well, and used to merge two seperate explodes. Reference the [function list](https://spark.apache.org/docs/2.3.0/api/sql/index.html).

_Note_: Some of the entires in `org` are "weird", i.e. made up of non-english letters and characters. Keep them. **DO NOT** edit any name in the original dataframe unless we specify. **DO NOT** drop any row unless there is a `null` value as stated before. This goes for the rest of the homework as well, unless otherwise specified.

In [ ]:
%%spark

# TODO: Create [raw_start_dates_sdf]

# YOUR CODE HERE


In [ ]:
%%spark
raw_start_dates_sdf.show(4)

### Step 1.4: The Fortuitous Formatting

There are two issues with the values in our `date` column. First, the values are saved as strings, not datetime types. This halts us from running functions such as `ORDER BY` or `GROUP BY` on common months or years. Second, some values do not have both month and year information or are in other languages. Your task is to filter out and clean the `date` column. We are interested in only those rows that have date in the following format "(month_name) (year)", e.g. "October 2010".

Create an sdf called `filtered_start_dates_sdf` from `raw_start_dates_sdf` with the `date` column filtered in the manner above. Keep only those rows with a start date between January 2000 to December 2011, inclusive. Ensure that any dates that are not in our desired format are ommitted. Drop any row that contains a `null` in either column. The format of the sdf is shown below:
```
+--------------------------+---------------+
|org                       |start_date     |
+--------------------------+---------------+
|Walt Disney World Resort  |2005-01-01     | 
|...                       |...            |
+--------------------------+---------------+
```
_Hint_: Refer to the [function list](https://spark.apache.org/docs/2.3.0/api/sql/index.html) to format the `date` column. In Spark SQL the date format we are interested in is `"MMM y"`.

_Note_: Spark will return the date in the format above, with the day as `01`. This is ok, since we are interested in the month and year each individual began working and all dates will have `01` as their day.

In [ ]:
%%spark

# TODO: Create [filtered_start_dates_sdf]


### Step 1.5 The Gregarious Grouping

We now want to collect the number of individuals that started in the same month and year for each company. Create an sdf called `start_dates_sdf` that has the total number of employees who began working at the same company on the same start date. The format of the sdf is shown below:

```
+--------------------------+---------------+---------------+
|org                       |start_date     |num_employees  |
+--------------------------+---------------+---------------+
|Walt Disney World Resort  |2005-01-01     |1              |
|...                       |...            |...            |
+--------------------------+---------------+---------------+
```

In [ ]:
%%spark

# TODO: Create [start_dates_sdf]
# YOUR CODE HERE


## Step 2: Hiring Trends Analysis

Now we will analyze `start_dates_sdf` to find monthly and annual hiring trends.

### Step 2.1: The Marvelous Months

Your task is to answer the question: "On average, what month do most employees start working?" Create an sdf called `monthly_hires_sdf` which contains the total number of employees that started working on a specific month, at any company and on any year. The `month` column should be of type `int`, i.e. 1-12. The format of the sdf is shown below:

```
+---------------+---------------+
|month          |num_employees  |
+---------------+---------------+
|1              |...            |
|2              |...            |
|3              |...            |
|...            |...            |
+---------------+---------------+
```

Find the month in which the most employees start working and save it's number as an integer to the variable `most_common_month`.

_Hint_: Be careful. The starts dates we have right now have both month and year. We only want the common months. See if you can find something in the [function list](https://spark.apache.org/docs/2.3.0/api/sql/index.html) that will help you do this.

In [ ]:
%%spark

# TODO: Create [monthly_hire_sdf] and find the most common month people were
# hired. Save its number as an integer to [most_common_month]


### Step 2.2: The Preposterous Percentages

The next question we will answer is "What is the percentage change in hires between 2010 and 2011 for each company?" Create an sdf called `percentage_change_sdf` that has the percentage change between 2010 and 2011 for each company. The sdf should look as follows:

```
+---------------------------+--------------------+
|org                        |percentage_change   |
+---------------------------+--------------------+
|Walt Disney World Resort   |12.3                |
|...                        |...                 |
+---------------------------+--------------------+
```

_Note_: A percentage change can be positive or negative depending 
on the difference between the two years.The formula for percent change is given below,

$$\text{% change} = \frac{P_f-P_i}{P_f} \times 100$$

Here, $P_f$ is the final element (in this case the number of hires in 2011) and $P_i$ is initial element (the number of hires in 2010).

_Hint_: This is a **difficult** question. We recommend using a combination of `GROUP BY` and `JOIN`. Keep in mind that operations between columns in SQL dataframes are often easier than those between rows. 

In [ ]:
%%spark

# TODO: Create [percentage_change_sdf]
# YOUR CODE HERE


## Step 3: Formatting the Training Data


Our overaching goal is to train a machine learning (ML) model that will use the monthly hiring trends of a company to predict a positive or negative gain in the company's stock in the first quarter of the following year. A ML model is trained on a set of observations. Each observation contains a set of features, `X`, and a label, `y`. The goal of the ML model is to create a function that takes any `X` as an input and outputs a predicted `y`. 

The machine learning model we will use is a [Random Forest Classifier](https://builtin.com/data-science/random-forest-algorithm). Each observation we will pass in will have 24 features (columns). These are the number of people hired from Jan to Dec and the company stock price on the last day of each month. The label will be the direction of the company's stock percentage change (positive, `1`, or negative, `-1`) in the first quarter of the following year. Each observation will correspond to a specified company's trends on a specified year. The format of our final training sdf is shown below. The first 26 columns define our observations, `X`, and the last column the label, `y`.
```
+----+-----+----------+---------+----------+----------+---------+----------+-------------+
|org |year |jan_hired |   ...   |dec_hired |jan_stock |   ...   |dec_stock |stock_result |
+----+-----+----------+---------+----------+----------+---------+----------+-------------+
|IBM |2008 |...       |   ...   |...       |...       |   ...   |...       |1            |
|IBM |2009 |...       |   ...   |...       |...       |   ...   |...       |-1           |
|... |...  |...       |   ...   |...       |...       |   ...   |...       |...          |
+----+-----+----------+---------+----------+----------+---------+----------+-------------+
```

_Note_: We will use the first three letters of each month in naming, i.e. `jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov, dec`



### Step 3.1: The Harmonious Hires

Your first task is to create the first half of the training table, i.e. the `jan_hired` through `dec_hired` columns. This will involve reshaping `start_dates_sdf`. Currently, `start_dates_sdf` has columns `org`, `start_date`, and `num_employees`. We want to group the rows together based on common `org` and years and create new columns for the number of employees that started working in each month of that year.

Create an sdf called `raw_hirings_for_training_sdf` that has for a single company and a single year, the number of hires in Jan through Dec, and the total number of hires that year. Note that for each company you will have several rows corresponding to years between 2000 and 2011. It is ok if for a given company you don't have a given year. However, ensure that for a given company and given year, each month column has an entry, i.e. if no one was hired the value should be `0`. The format of the sdf is shown below: 
```
+----+-----+----------+---------+----------+----------+
|org |year |jan_hired |   ...   |dec_hired |total_num |
+----+-----+----------+---------+----------+----------+
|IBM |2008 |...       |   ...   |...       |...       |
|IBM |2009 |...       |   ...   |...       |...       |
|... |...  |...       |   ...   |...       |...       |
+----+-----+----------+---------+----------+----------+
```
_Hint_: This is a **difficult** question. The tricky part is creating the additional columns of monthly hires, specifically when there are missing dates. In our dataset, if a company did not hire anybody in a given date, it will not appear in `start_dates_sdf`. We suggest you look into `CASE` and `WHEN` statements in the [function list](https://spark.apache.org/docs/2.3.0/api/sql/index.html).

In [ ]:
%%spark

# TODO: Create [raw_hire_train_sdf]
# YOUR CODE HERE


### Step 3.2: The Formidable Filters

Create an sdf called `hire_train_sdf` that contains all the observations in `raw_hire_train_sdf` with `total_num` greater than or equal to 10. The format of the sdf is shown below:

```
+----+-----+----------+---------+----------+----------+
|org |year |jan_hired |   ...   |dec_hired |total_num |
+----+-----+----------+---------+----------+----------+
|IBM |2008 |...       |   ...   |...       |...       |
|IBM |2009 |...       |   ...   |...       |...       |
|... |...  |...       |   ...   |...       |...       |
+----+-----+----------+---------+----------+----------+
```


In [ ]:
%%spark

# TODO: Create [hire_train_sdf]
# YOUR CODE HERE


### Step 3.3: The Stupendous Stocks

Now we are ready for the stock data. The stock data we will use is saved in the same S3 bucket as `linkedin.json`. Load the data into the EMR cluster. Run the cell below. ***You do not need to edit this cell, except changing dataset url***.

In [ ]:
%%spark
# TODO: modify the location to the place that you have the data. 
# Load stock data
raw_stocks_sdf = spark.read.format("csv") \
              .option("header", "true") \
              .load("s3a://xxx/stock_prices.csv")

# Creates SQL-accesible table
raw_stocks_sdf.createOrReplaceTempView('raw_stocks')

# Display the first 10 rows
query = '''SELECT *
           FROM raw_stocks'''
spark.sql(query).show(10)


Run the cell below to see the types of the columns in our data frame. These are not correct. We could have defined a schema when reading in data but we will handle this issue in another manner. You will do this in Step 3.4.2.

In [ ]:
%%spark 

# Print types of SDF
raw_stocks_sdf.dtypes

### Step 3.4 The Clairvoyant Cleaning

We now want to format the stock data set into the second half of the training table. We will then merge it with `hire_train` based off the common `org` and `year` fields. The formatting will consist of 4 steps. Actually, it is 5.

#### Step 3.4.1 The Ubiquitous UDF

The companies in our stock dataset are defined by their stock tickers. Thus, we would not be able to merge it with the `org` field in `hire_train_sdf`. We must convert them to that format. Often times when using Spark, there may not be a built-in SQL function that can do the operation we desired. Instead, we can create one on our own with a user-defined function (udf).

A udf is defined as a normal Python function and then registered to be used as a Spark SQL function. Your task is to create a udf, `TICKER_TO_NAME()` that will convert the ticker field in `raw_stocks` to the company's name. This will be done using the provided `ticker_to_name_dict` dictionary. We are only interested in the companies in that dictionary.

Fill out the function `ticker_to_name()` below. Then use `spark.udf.register()` to register it as a SQL function. The command is provided. ***You do not need to edit it***. Note, we have defined the udf as returning `StringType()`. Ensure that your function returns this. You must also deal with any potential `null` cases.

In [ ]:
%%spark

# TODO: Fill out [ticker_to_name()] and register it as a udf.
# YOUR CODE HERE

def ticker_to_name(ticker):

# Register udf as a SQL function. DO NOT EDIT
spark.udf.register("TICKER_TO_NAME", ticker_to_name, StringType())


#### Step 3.4.2: The Fastidious Filters

With our new `TICKER_TO_NAME()` function we will begin to wrangle `raw_stocks_sdf`.

Create an sdf called `filter_1_stocks_sdf` as follows. Convert all the ticker names in `raw_stocks_sdf` to the company names and save it as `org`. Next, convert the `date` field to a datetime type. As explained before this will help order and group the rows in future steps. Then, convert the type of the values in `closing_price` to `float`. This will take care of the `dtypes` issue we saw in Step 3.3.

Drop any company names that do not appear in `ticker_to_name_dict`. Keep any date between January 1st 2001 and December 4th 2012 inclusive, in the format shown below (note this is a datetime object not a string):

```
+----+------------+--------------+
|org |date        |closing_price |
+----+------------+--------------+
|IBM |2000-01-03  |...           |
|... |...         |...           |
+----+------------+--------------+
```
_Hint_: You will use a similar function to filter the dates as in Step 1.4. In Spark SQL the format for the `date` field in `raw_stocks_sdf` is `"yyyy-MM-dd"`.

In [ ]:
%%spark

# TODO: Create [filter_1_stocks_sdf]
# YOUR CODE HERE


#### Step 3.4.3: The Magnanimous Months

The data in `filter_1_stocks_sdf` gives closing prices on a daily basis. Since we are interested in monthly trends, we will only keep the closing price on the **last trading day of each month**.

Create an sdf `filter_2_stocks_sdf` that contains only the closing prices for the last trading day of each month. Note that a trading day is not simply the last day of each month, as this could be on a weekend when the market is closed . The format of the sdf is shown below:

```
+----+------------+--------------+
|org |date        |closing_price |
+----+------------+--------------+
|IBM |2000-01-31  |...           |
|... |...         |...           |
+----+------------+--------------+
```

  _Hint_: This is a **difficult** question. But if you made it this far, you're a star by now. It may be helpful to create an intermediate dataframe that will help you filter out the specific dates you desire.

In [ ]:
%%spark

# TODO: Create [filter_2_stocks_sdf]
# YOUR CODE HERE


#### Step 3.4.4: The Rambunctious Reshape

Now, we will begin to shape our dataframe into the format of the final training sdf.

Create an sdf `filter_3_stocks_sdf` that has for a single company and a single year, the closing stock price for the last trading day of each month in that year. This is similar to the table you created in Step 3.1. In this case since we cannot make a proxy for the closing price if the data is not avaliable, drop any rows containing any `null` values, in any column. The format of the sdf is shown below:

```
+----+-----+----------+---------+----------+
|org |year |jan_stock |   ...   |dec_stock |
+----+-----+----------+---------+----------+
|IBM |2008 |...       |   ...   |...       |
|IBM |2009 |...       |   ...   |...       |
|... |...  |...       |   ...   |...       |
+----+-----+----------+---------+----------+
```


In [ ]:
%%spark

# TODO: Create [filter_3_stocks_sdf]
# YOUR CODE HERE



#### Step 3.4.5: The Decisive Direction

The final element in our training set is the binary output for each case, i.e. the `y` label. 

Create an sdf `stocks_train_sdf` from `filter_3_stocks_sdf` with an additional column `direction`. This should be the direction of percentage change in the closing stock price, i.e. `1` for positive or `-1` for negative, in the first quarter of a given year. The quarter of a year begins in January and ends in April, inclusive. We want to know the percent change between these two months. Reference Step 2.2 for the percent change formula. The format of the sdf is shown below:

```
+----+-----+----------+---------+----------+-------------+
|org |year |jan_stock |   ...   |dec_stock |direction    |
+----+-----+----------+---------+----------+-------------+
|IBM |2008 |...       |   ...   |...       |1.0          |
|IBM |2009 |...       |   ...   |...       |-1.0         |
|... |...  |...       |   ...   |...       |...          |
+----+-----+----------+---------+----------+-------------+
```

In [ ]:
%%spark

# TODO: Create [stocks_train_sdf]
# YOUR CODE HERE


### Step 3.5: The Capricious Combination

Now that we have individually created the two halfs of our training data we will merge them together to create the final training sdf we showed in the beginning of Step 3.

Create an sdf called `training_sdf` in the format of the one shown at the beginning of Step 3. Note that in our definition for the `stock_result` column, the `stock_result` value for a particular year corresponds to the direction of the stock percentage change in the **following** year. For example, the stock_result in the `2008` row for `IBM` will contain the direction of IBM's stock in the first quarter of 2009. The format of the sdf is shown below:
```
+----+-----+----------+---------+----------+----------+---------+----------+-------------+
|org |year |jan_hired |   ...   |dec_hired |jan_stock |   ...   |dec_stock |stock_result |
+----+-----+----------+---------+----------+----------+---------+----------+-------------+
|IBM |2008 |...       |   ...   |...       |...       |   ...   |...       |-1.0         |
|IBM |2009 |...       |   ...   |...       |...       |   ...   |...       |1.0          |
|... |...  |...       |   ...   |...       |...       |   ...   |...       |...          |
+----+-----+----------+---------+----------+----------+---------+----------+-------------+
```

In [ ]:
%%spark

# TODO: Create [training_sdf]
# YOUR CODE HERE
